In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', 500)

import keggler as kg
from helpers import *
#import helpers

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm

import gc
gc.enable()

import warnings
warnings.simplefilter(action='ignore', category=Warning)

import os, psutil
import glob


import logging as logging
def ini_log(filename):
    logger = logging.getLogger(__name__)
    ## avoid multiple printouts due to same handlers added several times
    if not logger.handlers:
        logger.setLevel(logging.DEBUG)

        handlers = [#logging.StreamHandler(None), 
            logging.FileHandler(filename, 'a')
        ]

        fmt=logging.Formatter('%(asctime)-15s: %(levelname)s  %(message)s')
        for h in handlers:
            h.setFormatter(fmt)
            logger.addHandler(h)
    return logger
        
log = ini_log('out.log')

#PATH='data_mini/'
#prefix='_mini'

PATH='data/'
SUB_PATH='test_set/' #test_set #training_set

v_data='v2'
v_xtra='v4'
v_model=['m05i{}_6f'.format(i) for i in range(5)]

print(os.listdir(PATH))


['submissions', 'test_set', 'track_features', 'training_set']


In [2]:
half_1_logs = sorted(glob.glob(PATH + SUB_PATH + "outDD_"+v_data+"*.h5"))
half_1_xtra = sorted(glob.glob(PATH + SUB_PATH + "outDD_"+v_xtra+"*.h5"))
half_2_logs = sorted(glob.glob(PATH + SUB_PATH + "subDD_"+v_data+"*.h5"))
half_2_xtra = sorted(glob.glob(PATH + SUB_PATH + "subDD_"+v_xtra+"*.h5"))

In [3]:
half_1_logs[:3]

['data/test_set/outDD_v2_00.h5',
 'data/test_set/outDD_v2_01.h5',
 'data/test_set/outDD_v2_02.h5']

In [4]:
half_2_logs[:3]

['data/test_set/subDD_v2_00.h5',
 'data/test_set/subDD_v2_01.h5',
 'data/test_set/subDD_v2_02.h5']

In [5]:
n_trk = 5

# Read dataframes

In [6]:
from sklearn.preprocessing import LabelEncoder

def read_list_of_files(l1, l2, l1_xtr=None, l2_xtr=None, n_trk=1):
    df_1 = pd.concat([read_log(f) for f in l1], axis=0, ignore_index=True)
#     print(df_1.shape)
    if l1_xtr:
        df_xtr = pd.concat([read_log(f) for f in l1_xtr], axis=0, ignore_index=True)
        df_1 = pd.concat([df_1, df_xtr], axis=1)
        del df_xtr
    
    df_2 = pd.concat([read_log(f) for f in l2], axis=0, ignore_index=True)
#     print(df_2.shape)
    if l2_xtr:
        df_xtr = pd.concat([read_log(f) for f in l2_xtr], axis=0, ignore_index=True)
        df_2 = pd.concat([df_2, df_xtr], axis=1)
        
#     print(df_1.shape, df_2.shape)

#     df_1['session_id'] = df_1['session_id'].astype('category')
    le = LabelEncoder().fit(df_1['session_id'])
    df_1['session_id'] = le.transform(df_1['session_id'])
    df_1.rename({'not_skipped': 'skip_4'}, axis=1, inplace=True)

#     df_2['session_id'] = df_2['session_id'].astype('category')
    df_2['session_id'] = le.transform(df_2['session_id'])
    df_2.rename({'not_skipped': 'skip_4'}, axis=1, inplace=True)
    
    del le
    
#     display(df_2.head(30))
    
    # transform the data
    return get_XY((df_1, df_2), aggs, reset_index=True, 
                  list_musik_qualities_=list_musik_qualities,
                  aggs_music_qualities_=aggs_music_qualities,
                  i_=list(range(n_trk)),
                  aggs_trkvec_=aggs_trkvec, list_trkvec_=list_trkvec
                 )

In [7]:
def pred_series_of_lists(list_preds, y_length, i_2fill=-2):
    # transform predictions into a dataframe
    tmp_preds_constant_mdl = pd.DataFrame({'pred_{}'.format(i): list_preds[i] for i in range(len(list_preds))}).astype(np.uint8)
    # add a column with the residual desired length of complete session
    tmp_preds_constant_mdl['len'] = y_length.values - len(list_preds)
    # create a series with lists
    series_of_lists = tmp_preds_constant_mdl.apply(lambda x: x.iloc[:-1].tolist() + [x.iloc[i_2fill]]*x['len'], axis=1)
    del tmp_preds_constant_mdl
    return series_of_lists

def make_predictions(l1, l2, l1_xtr=None, l2_xtr=None, n_trk=1):
    X_trn, [target_length], X_trk = read_list_of_files(l1, l2, l1_xtr=l1_xtr, l2_xtr=l2_xtr, n_trk=n_trk)

    l_prob = []
    for i_ in tqdm(list(range(n_trk))):
        X = pd.concat([X_trn, X_trk[i_]], axis=1)
        if len(l_prob) > 0:
            X = pd.concat([X, pd.DataFrame({'pred_trk{}'.format(j): l_prob[j] 
                                            for j in range(len(l_prob))
                                           })],
                          axis=1)
        import joblib
        prob_pred = np.zeros(shape=(len(X),))
        for v in v_model:
            mfn = 'models/model_{}_{}_{}.pkl'.format(v_data, v, i_)
    #         print(mfn)
            mdl = joblib.load(mfn)
            prob_pred += mdl.predict_proba(X)[:,1]

        prob_pred /= len(v_model)

        l_prob.append(prob_pred)
        
#     print([p>0.50 for p in l_prob])
    return pred_series_of_lists([p>0.50 for p in l_prob], target_length, i_2fill=-2)

In [8]:
#!rm data/submissions/sub_v1.txt

In [ ]:
n_part = 66

for i, (a1, a2, a1_xtr, a2_xtr) in tqdm(enumerate(zip(np.array(half_1_logs).reshape(n_part,-1),
                  np.array(half_2_logs).reshape(n_part,-1),
                  np.array(half_1_xtra).reshape(n_part,-1),
                  np.array(half_2_xtra).reshape(n_part,-1),  
                 ))):
#     print(i)
#     print(a1, a2)
#     print(a1_xtr, a2_xtr)
    x1 = make_predictions(a1.tolist(), a2.tolist(), 
                          l1_xtr=a1_xtr.tolist(), l2_xtr=a2_xtr.tolist(), 
                          n_trk=10)
    gc.collect()
    log.info(psutil.Process(os.getpid()).memory_info().rss / 1024**2)
    
    x1.apply(lambda l: ''.join(map(str,l))).to_csv(PATH+'submissions/sub_'+v_data+v_xtra+'_'+v_model[0]+'.txt', index=False, mode='a')
    gc.collect()
    log.info(psutil.Process(os.getpid()).memory_info().rss / 1024**2)
#     if i >=1:
#         break

0it [00:00, ?it/s]
100%|██████████| 10/10 [02:05<00:00, 11.29s/it]
1it [05:18, 318.16s/it]
100%|██████████| 10/10 [02:21<00:00, 12.74s/it]
2it [11:14, 329.48s/it]
100%|██████████| 10/10 [02:20<00:00, 12.69s/it]
3it [17:13, 338.35s/it]
100%|██████████| 10/10 [02:23<00:00, 13.15s/it]
4it [23:17, 346.13s/it]
100%|██████████| 10/10 [02:13<00:00, 11.45s/it]
5it [29:03, 346.07s/it]
100%|██████████| 10/10 [02:06<00:00, 11.46s/it]
6it [34:38, 342.79s/it]
100%|██████████| 10/10 [02:51<00:00, 15.50s/it]
7it [41:22, 361.16s/it]
100%|██████████| 10/10 [02:40<00:00, 14.62s/it]
8it [48:03, 373.13s/it]
100%|██████████| 10/10 [03:07<00:00, 16.69s/it]
9it [55:49, 400.96s/it]
100%|██████████| 10/10 [03:10<00:00, 17.08s/it]
10it [1:03:43, 422.86s/it]
100%|██████████| 10/10 [1:49:21<00:00, 332.34s/it]
11it [2:56:38, 2328.65s/it]
100%|██████████| 10/10 [01:30<00:00,  8.21s/it]
12it [3:00:47, 1704.54s/it]
100%|██████████| 10/10 [01:39<00:00,  9.11s/it]
13it [3:05:13, 1273.02s/it]
100%|██████████| 10/10 [01:

100%|██████████| 10/10 [01:21<00:00,  7.37s/it]
15it [3:12:53, 740.20s/it]
100%|██████████| 10/10 [01:27<00:00,  7.96s/it]
16it [3:16:52, 589.76s/it]
100%|██████████| 10/10 [01:28<00:00,  8.01s/it]
17it [3:20:52, 484.96s/it]
100%|██████████| 10/10 [01:44<00:00, 10.13s/it]
18it [3:25:26, 421.54s/it]
100%|██████████| 10/10 [02:01<00:00, 10.90s/it]
19it [3:30:38, 388.75s/it]
100%|██████████| 10/10 [02:18<00:00, 12.52s/it]
20it [3:36:29, 377.36s/it]
100%|██████████| 10/10 [02:12<00:00, 12.09s/it]
21it [3:42:06, 365.43s/it]
100%|██████████| 10/10 [01:53<00:00, 10.21s/it]
22it [3:46:55, 342.41s/it]
100%|██████████| 10/10 [02:05<00:00, 11.35s/it]
23it [3:52:12, 334.76s/it]
100%|██████████| 10/10 [02:08<00:00, 11.55s/it]
24it [3:57:36, 331.45s/it]
100%|██████████| 10/10 [1:03:21<00:00, 551.84s/it]
25it [5:04:09, 1430.17s/it]
100%|██████████| 10/10 [01:43<00:00,  9.42s/it]
26it [5:08:41, 1082.44s/it]
100%|██████████| 10/10 [01:38<00:00,  8.94s/it]
27it [5:12:22, 824.07s/it] 
100%|██████████| 10

100%|██████████| 10/10 [01:22<00:00,  7.49s/it]
29it [5:18:41, 499.59s/it]
100%|██████████| 10/10 [01:32<00:00,  8.33s/it]
30it [5:22:01, 409.76s/it]
100%|██████████| 10/10 [01:38<00:00,  8.71s/it]
31it [5:25:31, 349.81s/it]
100%|██████████| 10/10 [01:33<00:00,  8.43s/it]
32it [5:28:51, 304.86s/it]
100%|██████████| 10/10 [01:35<00:00,  8.55s/it]
33it [5:32:16, 275.02s/it]
100%|██████████| 10/10 [01:42<00:00,  9.28s/it]
34it [5:35:59, 259.46s/it]
100%|██████████| 10/10 [01:32<00:00,  8.37s/it]
35it [5:39:17, 240.85s/it]
100%|██████████| 10/10 [01:24<00:00,  7.55s/it]
36it [5:42:16, 222.40s/it]
100%|██████████| 10/10 [01:37<00:00,  8.57s/it]
37it [5:45:46, 218.62s/it]
100%|██████████| 10/10 [01:42<00:00,  9.32s/it]
38it [5:49:28, 219.58s/it]
100%|██████████| 10/10 [01:41<00:00,  9.12s/it]
39it [5:53:08, 219.70s/it]
100%|██████████| 10/10 [01:31<00:00,  8.33s/it]
40it [5:56:29, 214.16s/it]
100%|██████████| 10/10 [01:38<00:00,  8.95s/it]
41it [6:00:10, 216.13s/it]
100%|██████████| 10/10 [0

100%|██████████| 10/10 [01:18<00:00,  7.26s/it]
43it [6:06:10, 196.93s/it]
100%|██████████| 10/10 [01:30<00:00,  8.23s/it]
44it [6:09:25, 196.48s/it]
100%|██████████| 10/10 [01:45<00:00,  9.17s/it]
45it [6:13:00, 202.01s/it]
100%|██████████| 10/10 [01:43<00:00,  9.56s/it]
46it [6:16:50, 210.34s/it]
100%|██████████| 10/10 [01:36<00:00,  9.02s/it]
47it [6:20:24, 211.46s/it]
100%|██████████| 10/10 [01:49<00:00,  9.96s/it]
48it [6:24:33, 222.63s/it]
100%|██████████| 10/10 [01:34<00:00,  8.40s/it]
49it [6:28:01, 218.29s/it]
100%|██████████| 10/10 [01:35<00:00,  8.44s/it]
50it [6:32:10, 227.59s/it]
100%|██████████| 10/10 [01:29<00:00,  7.95s/it]
51it [6:35:25, 217.74s/it]
100%|██████████| 10/10 [01:24<00:00,  7.67s/it]
52it [6:38:35, 209.38s/it]
 70%|███████   | 7/10 [01:03<00:26,  8.74s/it]

In [ ]:
!ls -ltr data/submissions/

In [ ]:
!wc -l
